# Avance - Sistema de Recomendacion
## Feedback Implícito
## Análisis de Grandes Volúmenes de Datos

En la entrega pasada analizamos un dataset de reviews de Steam que tenian los reviews de miles de aplicaciones sin estar asociadas a un Steam_ID específico, es decir, que solo es posible establecer un algoritmo de recomendación hasta el promedio bayesiano sin proceder a filtrado colaborativo. Se consiguió una base de datos mayor y con más puntos de información que incluyen el Steam_ID de tal forma que ahora es posible conocer los juegos que un usuario específico está reseñando en la plataforma para recomendar a otros usuarios con gustos similares.

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install implicit
from fuzzywuzzy import process

import implicit
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

C:\Users\DanielGuzmánÁvila\AppData\Roaming\Python\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### El primer paso consiste en hacer los dataframes (uno de los ratings y otro de los juegos) y hacer una limpieza para eliminar NaN, Null y reviews con sospecha de fraude

In [ ]:
# Abre el archivo original
steam_ratings = pd.read_csv("archive/steam_reviews.csv", usecols=['app_id', 'app_name', 'author.playtime_forever', 'author.steamid'])
# Tira los valores nulos y na
steam_ratings = steam_ratings.dropna()
# Solo nos traemos informacion de jugadores que hayan jugado más de tres horas
steam_ratings = steam_ratings[steam_ratings['author.playtime_forever'] >= 3]

# Creamos una lista con el ID de steam y el nombre del juego
steam_games = steam_ratings[['app_id', 'app_name']].drop_duplicates()
# Esta ultima línea no se usa normalmente, solo es para guardar la lista de juegos como archivo y revisar el output en Excel
#steam_games.to_csv("archive/steam_games.csv", index=False)

In [ ]:
steam_ratings.head()

,app_id,app_name,author.steamid,author.playtime_forever
0,292030,The Witcher 3: Wild Hunt,76561199095369542,1909.0
1,292030,The Witcher 3: Wild Hunt,76561198949504115,2764.0
2,292030,The Witcher 3: Wild Hunt,76561199090098988,1061.0
3,292030,The Witcher 3: Wild Hunt,76561199054755373,5587.0
4,292030,The Witcher 3: Wild Hunt,76561199028326951,217.0


In [ ]:
steam_games.head()

,app_id,app_name
0,292030,The Witcher 3: Wild Hunt
469395,70,Half-Life
526715,240,Counter-Strike: Source
644796,420,Half-Life 2: Episode Two
668296,620,Portal 2


In [ ]:
steam_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21746584 entries, 0 to 21747370
Data columns (total 4 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   app_id                   int64  
 1   app_name                 object 
 2   author.steamid           int64  
 3   author.playtime_forever  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 829.6+ MB


In [ ]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 315 entries, 0 to 21696272
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   app_id    315 non-null    int64 
 1   app_name  315 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.4+ KB


In [ ]:
print(steam_ratings.isna().sum())
print('##################')
print(steam_games.isna().sum())

app_id                     0
app_name                   0
author.steamid             0
author.playtime_forever    0
dtype: int64
##################
app_id      0
app_name    0
dtype: int64


* Descripción de los campos

Steam Reviews Dataset 2021. (2021, 25 enero). Kaggle. https://www.kaggle.com/datasets/najzeko/steam-reviews-2021

| Campo| Descripción|
|------|------|                
| app_id | ID en steam del aplicativo                         
| app_name | Nombre del aplicativo                          
| author.steamid | Cuenta anonimizada de steam de quien deja la reseña                        
| author.playtime_forever | Tiempo total jugado                

* Para este ejercicio vamos a considerar que las horas totales jugadas son una especie de ranking implícito. Ejemplo: Si un jugador jugó 50 horas Rocket League y otro 10 horas el primer jugador tiene una mejor opinión de Rocket League que el segundo
* Similar al dataset de Movielens existen relanzamientos, sin embargo, la gran mayoría de los relanzamientos incluyen mejoras en el funcionamiento del aplicativo o son una versión completamente distinta con la misma historia pero diseñado para hardware más nuevo. Por esta razón no vamos a eliminar títulos duplicados

### El segundo paso consiste en hacer una matriz dispersa con los jugadores y los juegos como dimensiones, la intersección sería el tiempo jugado total

In [ ]:
def create_X(df: pd.DataFrame):

    N = df['author.steamid'].nunique()
    M = df['app_id'].nunique()

    user_mapper = dict(zip(np.unique(df["author.steamid"]), list(range(N))))
    game_mapper = dict(zip(np.unique(df["app_id"]), list(range(M))))

    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["author.steamid"])))
    game_inv_mapper = dict(zip(list(range(M)), np.unique(df["app_id"])))

    user_index = [user_mapper[i] for i in df['author.steamid']]
    game_index = [game_mapper[i] for i in df['app_id']]

    X = csr_matrix((df["author.playtime_forever"], (game_index, user_index)), shape=(M, N))

    return X, user_mapper, game_mapper, user_inv_mapper, game_inv_mapper

In [ ]:
X, user_mapper, game_mapper, user_inv_mapper, game_inv_mapper = create_X(steam_ratings)
print(X)

  (0, 0)	407.0
  (0, 16)	8803.0
  (0, 61)	38.0
  (0, 246)	842.0
  (0, 361)	89.0
  (0, 372)	252.0
  (0, 396)	994.0
  (0, 408)	4636.0
  (0, 749)	80.0
  (0, 780)	838.0
  (0, 795)	1063.0
  (0, 806)	7.0
  (0, 829)	475.0
  (0, 916)	4438.0
  (0, 982)	293528.0
  (0, 1053)	12523.0
  (0, 1075)	902.0
  (0, 1140)	139026.0
  (0, 1189)	296498.0
  (0, 1208)	1165.0
  (0, 1217)	389.0
  (0, 1226)	2269.0
  (0, 1390)	4080.0
  (0, 1461)	71308.0
  (0, 1464)	12.0
  :	:
  (314, 12359517)	188.0
  (314, 12360071)	291.0
  (314, 12360078)	1556.0
  (314, 12361040)	817.0
  (314, 12361146)	198.0
  (314, 12361706)	603.0
  (314, 12361925)	1268.0
  (314, 12366182)	196.0
  (314, 12368945)	46.0
  (314, 12376723)	1050.0
  (314, 12377191)	263.0
  (314, 12379227)	127.0
  (314, 12380001)	45.0
  (314, 12380595)	220.0
  (314, 12392580)	196.0
  (314, 12396667)	1286.0
  (314, 12397064)	1636.0
  (314, 12399051)	383.0
  (314, 12399187)	52.0
  (314, 12399871)	315.0
  (314, 12402077)	242.0
  (314, 12403938)	60.0
  (314, 12404287)	27

### A continuación escribimos las funciones para construir los mapas de los nombres de los juegos y busqueda de strings similares  

In [ ]:
def game_finder(title):
    all_titles = steam_games['app_name'].tolist()
    closest_match = process.extractOne(title,all_titles)
    return closest_match[0]

game_title_mapper = dict(zip(steam_games['app_name'], steam_games['app_id']))
game_title_inv_mapper = dict(zip(steam_games['app_id'], steam_games['app_name']))

def get_game_index(title):
    fuzzy_title = game_finder(title)
    game_id = game_title_mapper[fuzzy_title]
    game_idx = game_mapper[game_id]
    return game_idx


def get_game_title(game_idx):
    game_id = game_inv_mapper[game_idx]
    title = game_title_inv_mapper[game_id]
    return title


### Probamos con un título aproximado de un juego

In [ ]:
print(get_game_index('witcher'))
print(get_game_title(79))


79
The Witcher 3: Wild Hunt


### El siguiente paso es entrenar el modelo  

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(X.T.tocsr())

C:\Users\DanielGuzmánÁvila\AppData\Roaming\Python\Python311\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 10 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
C:\Users\DanielGuzmánÁvila\AppData\Roaming\Python\Python311\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

### Podemos hacer una prueba con Rocket League

In [ ]:
game_of_interest = 'rocket league'

game_index = get_game_index(game_of_interest)
related = model.similar_items(game_index)
related

(array([ 54,  17,  16, 196,   1, 208, 101,  52,   5,  96]),
 array([1.        , 0.8057992 , 0.8010802 , 0.7970826 , 0.79685163,
        0.78789794, 0.78069323, 0.76943725, 0.7691637 , 0.7667666 ],
       dtype=float32))

In [ ]:
print(f"Por que jugaste {game_finder(game_of_interest)} te pueden interesar los siguientes juegos:")
for t, r in zip(related[0], related[1]):

    recommended_title = get_game_title(t)
    if recommended_title != game_finder(game_of_interest):
        print(recommended_title)

Por que jugaste Rocket League te pueden interesar los siguientes juegos:
Terraria
The Elder Scrolls V: Skyrim
PLAYERUNKNOWN'S BATTLEGROUNDS
Counter-Strike: Source
Total War: WARHAMMER II
Tom Clancy's Rainbow Six Siege
The Binding of Isaac: Rebirth
Garry's Mod
ARK: Survival Evolved


### Hagamos una prueba Rainbow 6 a ver si nos salen titulos similares

In [ ]:
game_of_interest = 'rainbow six'

print(f"Por que jugaste {game_finder(game_of_interest)} te pueden interesar los siguientes juegos:")
for t, r in zip(related[0], related[1]):

    recommended_title = get_game_title(t)
    if recommended_title != game_finder(game_of_interest):
        print(recommended_title)

Por que jugaste Tom Clancy's Rainbow Six Siege te pueden interesar los siguientes juegos:
Rocket League
Terraria
The Elder Scrolls V: Skyrim
PLAYERUNKNOWN'S BATTLEGROUNDS
Counter-Strike: Source
Total War: WARHAMMER II
The Binding of Isaac: Rebirth
Garry's Mod
ARK: Survival Evolved


### Como último paso generamos recomendaciones por usuario

In [ ]:
user_id = 76561198149550625

user_ratings = steam_ratings[steam_ratings['author.steamid']==user_id].merge(steam_games[['app_id', 'app_name']])
user_ratings = user_ratings.sort_values('author.playtime_forever', ascending=False)
print(f"El numero de juegos reseñados por el Usuario {user_id} es de: {user_ratings['app_id'].nunique()}")

El numero de juegos reseñados por el Usuario 76561198149550625 es de: 6


In [ ]:
X_t = X.T.tocsr()
user_idx = user_mapper[user_id]
recommendations = model.recommend(user_idx, X_t[user_idx])
recommendations

for t, r in zip(recommendations[0], recommendations[1]):
    recommended_title = get_game_title(t)
    print(recommended_title)

Assassin's Creed Odyssey
MORDHAU
The Forest
PAYDAY 2
Counter-Strike: Source
Factorio
ARK: Survival Evolved
Rust
Sid Meier's Civilization VI
Age of Empires II (2013)
